In [6]:
import pandas as pd
import geopandas as gpd
import os
import glob
import random
import time

# ruta base donde estan las carpetas que contienen los csvs de https://github.com/LMCappelletti/data.git

directorio_dataFundar = '/home/capuccino/Desktop/TrabajoFundar/Argendata_renaming/data/data_argendata'


# dict para guardar los nombres de todas las columnas de todos los csvs de todas las carpeta dataFundar
dict_ALLcsv_ALLcolumnas = {}

# csv que contiene el esquema de encoding y delimitador para cada dataset registrado al momento
# este CSV tiene las columnas: 'dataset', 'detected_encoding' y 'delimiter'
archivo_encoding = '/home/capuccino/Desktop/TrabajoFundar/Argendata_renaming/data/input/esquemaEncoding_por_dataset.csv'
df_enc = pd.read_csv(archivo_encoding)

# zip toma iterables y los "empareja" elemento a elemento, creando un iterador de tuplas
# creo diccionario que asocia a cada dataset (archivo CSV) su detected_encoding y delimiter
encoding_delim_dict = dict(zip(df_enc['dataset'], zip(df_enc['detected_encoding'], df_enc['delimiter'])))


# lista para almacenar archivos que TAL VEZ no figuran en esquemaEncoding_por_dataset.csv
archivos_no_encontrados = []

# recorrer todas las carpetas y subcarpetas
# recorre todos los directorios y subdirectorios desde la ruta base, obteniendo archivos en cada carpeta

# root: la ruta actual del directorio que se está recorriendo (la carpeta con todos los topicos)
# dirs: lista con los nombres de las subcarpetas que se encuentran en la carpeta actual
# files: lista con los nombres de los archivos en la carpeta actual

for root, dirs, files in os.walk(directorio_dataFundar): 
    
    # recorre todos las carpetas y subcarpetas desde la ruta base, obteniendo archivos en cada carpeta
    for file in files:
        file_dir = os.path.join(root, file)
        
        # solo considerar archivos con extensión .csv
        if file.endswith('.csv'):
            if file in encoding_delim_dict:
                encoding, delim = encoding_delim_dict[file]
                # solo leemos la cabecera (fila 0) usando el encoding y delimitador determinados, segun el input de delimitadores generado en el proceso QA
                df = pd.read_csv(file_dir, nrows = 0, encoding = encoding, delimiter = delim)
                dict_ALLcsv_ALLcolumnas[file] = list(df.columns)
            else:
                archivos_no_encontrados.append(file)

# lista de archivos que no figuran en esquemaEncoding_por_dataset.csv
print('Archivos que no figuran en esquemaEncoding_por_dataset.csv:')
print(archivos_no_encontrados)


UnicodeDecodeError: 'ascii' codec can't decode byte 0xc3 in position 323: ordinal not in range(128)

In [3]:
print(dict_ALLcsv_ALLcolumnas)

{'ISA_composicion-ipcf_it4.csv': ['year', 'semestre', 'genero', 'proporcion'], 'ISA_tipo_empleo_i1.csv': ['ano', 'variable', 'valor'], 'ISA_salarios_region_i2.csv': ['ano_trim', 'variable', 'valor'], 'ISA_salario_real_i1.csv': ['ano', 'variable', 'valor'], 'ISA_ipcf_it1.csv': ['year', 'semestre', 'ipcf_index'], 'ISA_regiones-ipcf_it1.csv': ['year', 'semestre', 'region', 'deflactor', 'indice'], 'ISA_salario_real_i3.csv': ['ano', 'salario'], 'salario_real_ppa2017_ceped.csv': ['iso3', 'anio', 'salario_real_ppa_consumo_privado_2017'], 'ISA_composicion-ipcf_it3.csv': ['year', 'semestre', 'edad_jefe', 'fuente', 'proporcion'], 'ISA_salarios_mundo_i2.csv': ['ano', 'variable', 'valor'], 'ISA_salarios_mundo_i1.csv': ['pais', 'salariohorario'], 'ISA_salarios_region_i1.csv': ['region', 'variable', 'valor'], 'ISA_composicion-ipcf_it5.csv': ['year', 'semestre', 'fuente', 'indice'], 'ISA_regiones-ipcf_it2.csv': ['year', 'semestre', 'region', 'indice'], 'ISA_composicion-ipcf_it1.csv': ['year', 'semest

In [4]:
len(dict_ALLcsv_ALLcolumnas)

337

In [ ]:
import os
import chardet #type: ignore
import csv

def detect_encoding_delimiter(file_path, sample_size = 100000):
    '''
    detecta el encoding y delimitador de un archivo .csv
    
    lee un fragmento del archivo (por defecto, 100000 bytes) y se utiliza:
      - chardet para detectar el encoding.
      - csv.Sniffer para detectar el delimitador.
      
    retorna una tupla (encoding, delimiter).
    '''
    with open(file_path, 'rb') as f:
        sample = f.read(sample_size)
    
    detected = chardet.detect(sample)
    encoding = detected['encoding']
    
    try:
        sample_text = sample.decode(encoding)
    except Exception:
        sample_text = sample.decode(encoding, errors='replace')
    
    sniffer = csv.Sniffer()
    try:
        dialect = sniffer.sniff(sample_text)
        delimiter = dialect.delimiter
    except Exception:
        delimiter = ','  # valor por defecto
    
    return encoding, delimiter

# ruta base donde estan las carpetas que contienen los csvs de https://github.com/LMCappelletti/data.git
directorio_dataFundar = '/home/capuccino/Desktop/TrabajoFundar/Argendata_renaming/data/data_argendata'

# dict para guardar los nombres de todas las columnas de todos los csvs de todas las carpeta dataFundar
dict_ALLcsv_ALLcolumnas = {}

# Lista para almacenar la información de cada CSV: (nombre del CSV, encoding detectado, delimitador detectado)
info_csv = []

# Recorrer todas las carpetas y subcarpetas
for root, dirs, files in os.walk(directorio_dataFundar):
    for file in files:
        file_dir = os.path.join(root, file)
        if file.endswith('.csv'):
            encoding, delim = detect_encoding_delimiter(file_dir)
            info_csv.append((file, encoding, delim))
            # lee fila 0 usando el encoding y delimitador detectados
            df = pd.read_csv(file_dir, nrows = 0, encoding = encoding, delimiter = delim, encoding_errors = 'replace')
            dict_ALLcsv_ALLcolumnas[file] = list(df.columns)

# Generar un DataFrame con la información de cada CSV
df_info = pd.DataFrame(info_csv, columns=['csv', 'encoding', 'delimiter'])

print(df_info)
print(dict_ALLcsv_ALLcolumnas)
print(len(dict_ALLcsv_ALLcolumnas))

output_path_encodindDelimiters = '/home/capuccino/Desktop/TrabajoFundar/Argendata_renaming/data/'
df_info.to_csv('data_encodings_and_delimiters.csv', index = False)



                                                  csv encoding delimiter
0                        ISA_composicion-ipcf_it4.csv    ascii         ,
1                              ISA_tipo_empleo_i1.csv    utf-8         ,
2                          ISA_salarios_region_i2.csv    ascii         ,
3                             ISA_salario_real_i1.csv    ascii         ,
4                                    ISA_ipcf_it1.csv    ascii         ,
..                                                ...      ...       ...
332                  tasa_actividad_por_pais_anio.csv    utf-8         ,
333                     tasa_participacion_censos.csv    ascii         ,
334     tiempo_social_trabajo_sexo_niveleducativo.csv    utf-8         ,
335  tasa_empleo_por_franja_etaria_anio_provincia.csv    utf-8         ,
336                      tipo_trabajo_no_rem_sexo.csv    utf-8         ,

[337 rows x 3 columns]
{'ISA_composicion-ipcf_it4.csv': ['year', 'semestre', 'genero', 'proporcion'], 'ISA_tipo_empleo_i1.c